In [3]:
import torch
import  torch .nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs=5
num_classes=10

batch_size=100

learning_rate=0.001

train_dataset = torchvision.datasets.MNIST(root='D:/data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

train_loader = DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='D:/data',
                                          train=False,
                                          transform=transforms.ToTensor())

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                        shuffle=False)
class ResBlock(nn.Module):
    def __init__(self,chanels):
        super(ResBlock, self).__init__()
        self.channels=chanels
        self.conv1=nn.Conv2d(
            chanels,chanels,kernel_size=3,padding=1
        )
        self.conv2 = nn.Conv2d(
            chanels, chanels, kernel_size=3, padding=1
        )
    def forward(self,x):
        y=F.relu(self.conv1(x))
        y=self.conv2(y)
        return F.relu(x+y)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1=nn.Conv2d(1,16,kernel_size=5)
        self.conv2=nn.Conv2d(16,32,kernel_size=5)
        self.mp=nn.MaxPool2d(2)
        
        self.block1=ResBlock(16)
        self.block2=ResBlock(32)
        
        self.fc=nn.Linear(512,10)
        
    def forward(self,x):
        in_size=x.size(0)
        x=self.mp(F.relu((self.conv1(x))))
        x=self.block1(x)

        x = self.mp(F.relu((self.conv2(x))))
        x = self.block2(x)
        
        x=x.view(in_size,-1)
        
        x=self.fc(x)
        
        return x


In [17]:
conv_model=Net().to(device)

criterion=nn.CrossEntropyLoss()

opt=torch.optim.Adam(conv_model.parameters(),lr=learning_rate)

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images=images.to(device)
        labels=labels.to(device)

        out_puts=conv_model(images)

        loss=criterion(out_puts,labels)

        opt.zero_grad()

        loss.backward()

        opt.step()
        if (i + 1) % 100 == 0 :
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
conv_model.eval()

with torch.no_grad():
    correct=0
    total=0
    for images_,labels_ in (test_loader):
        images_=images_.to(device)
        labels_=labels_.to(device)
        out_puts_=conv_model(images_)
        _,predicted=torch.max(out_puts_.data,dim=1)
        total+=labels_.size(0)
        correct+=(predicted==labels_).sum().item()
        print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Epoch [1/5], Step [100/600], Loss: 1.4803
Epoch [1/5], Step [200/600], Loss: 0.6912
Epoch [1/5], Step [300/600], Loss: 0.4603
Epoch [1/5], Step [400/600], Loss: 0.3953
Epoch [1/5], Step [500/600], Loss: 0.2507
Epoch [1/5], Step [600/600], Loss: 0.2420
Epoch [2/5], Step [100/600], Loss: 0.1004
Epoch [2/5], Step [200/600], Loss: 0.1004
Epoch [2/5], Step [300/600], Loss: 0.1205
Epoch [2/5], Step [400/600], Loss: 0.1452
Epoch [2/5], Step [500/600], Loss: 0.1084
Epoch [2/5], Step [600/600], Loss: 0.2026
Epoch [3/5], Step [100/600], Loss: 0.0365
Epoch [3/5], Step [200/600], Loss: 0.1433
Epoch [3/5], Step [300/600], Loss: 0.2576
Epoch [3/5], Step [400/600], Loss: 0.0396
Epoch [3/5], Step [500/600], Loss: 0.1520
Epoch [3/5], Step [600/600], Loss: 0.0647
Epoch [4/5], Step [100/600], Loss: 0.0362
Epoch [4/5], Step [200/600], Loss: 0.1225
Epoch [4/5], Step [300/600], Loss: 0.1187
Epoch [4/5], Step [400/600], Loss: 0.1245
Epoch [4/5], Step [500/600], Loss: 0.0151
Epoch [4/5], Step [600/600], Loss:

In [14]:
train_dataset.classes

['0 - zero',
 '1 - one',
 '2 - two',
 '3 - three',
 '4 - four',
 '5 - five',
 '6 - six',
 '7 - seven',
 '8 - eight',
 '9 - nine']